## `Материалы кафедры ММП факультета ВМК МГУ. Введение в эффективные системы глубокого обучения.`

## `Задание 04. Работа с большими моделями`

#### Фамилия, имя:

Дата выдачи: <span style="color:red">__28.10.2025 21:30__</span>.

Мягкий дедлайн: <span style="color:red">__11.11.2025 11:11__</span>.

Стоимость: __10 баллов__ (основная часть заданий) + __3 балла__ (дополнительные задания).

<span style="color:red">__В ноутбуке все клетки должны выполняться без ошибок при последовательном их выполнении.__</span>

#### `Москва, 2025`

Авторы задания: `@sir_rois` и `@jserdyuk`

## `Постановка задачи`

В данном задании вы научитесь обучать модель при помощи LoRA, а также реализуете CPU offloading.

Используемая в задании модель Qwen/Qwen2.5-0.5B

Используемый датасет - GSM8K (набор несложных математических задач с решением и ответом)

## `Необходимая теория`

### `LoRA (Low-Rank Adaptation)`
**LoRA** — это метод адаптации больших языковых моделей, который позволяет обучать только небольшие низкоранговые матрицы вместо всех весов модели.  
- Позволяет **экономить память и время обучения**.  
- Подходит для **тонкой настройки (fine-tuning)** моделей без переобучения всех параметров.  
- Обучение ограничивается несколькими адаптационными матрицами, а основная модель остаётся неизменной.

Идея простая: вместо того чтобы обновлять все веса слоя `W`, мы добавляем **низкоранговую поправку**:

$$
W' = W + \Delta W, \quad \Delta W = B A
$$

где:  
- $A \in \mathbb{R}^{d_\text{in} \times r}$ и$ B \in \mathbb{R}^{r \times d_\text{out}}$ — **малые матрицы для адаптации**,  
- $r \ll d_\text{in}, d_\text{out}$ — низкий ранг,  
- дополнительно можно использовать **масштабирование** и **dropout** для стабилизации обучения.



### `CPU Offloading`
**CPU Offloading** — это техника, когда часть модели (например, слои или параметры) хранится и обрабатывается на **CPU**, а не на GPU.  
- Позволяет работать с **очень большими моделями**, которые не помещаются целиком на GPU.  
- GPU используется только для активного вычисления, что снижает требования к памяти GPU.

### `CPU Offloading with Overlap`

**CPU Offloading with Overlap** — это усовершенствованная техника CPU Offloading, при которой **перемещение слоев между CPU и GPU происходит параллельно с вычислениями на GPU**.  

- **Обычный CPU Offloading:** слой загружается на GPU → выполняется вычисление → выгружается обратно на CPU.  
  Это может создавать паузы, потому что GPU ждёт загрузки слоя.  

- **С Overlap:**  
  - Пока GPU вычисляет текущий слой, **следующий слой уже подгружается с CPU**.  
  - Это **сокращает простой GPU**, повышая скорость обработки.  

Идея в том, чтобы **сглаживать задержки при работе с большими моделями**, которые не помещаются целиком на GPU.

<span style="color:red">ВНИМАНИЕ!</span>.
 Ячейки, в которых ожидается ваш письменный ответ, помечены (❓). Перед отправкой ноутбука проверьте, что вы ответили на все вопросы 

## `Часть 1. Обучение LoRA (2 балла)`

Начнем с импорта необходимых библиотек, загрузки модели, разбиения датасета на трейн, валидацию и тест.

In [1]:
import re
from tqdm import tqdm
import torch
from torch import nn
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch.nn.functional as F

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

dataset = load_dataset("gsm8k", "main", split="train")

train_eval_test = dataset.train_test_split(test_size=0.1, seed=42)
eval_test_dataset = train_eval_test['test']
train_dataset = train_eval_test['train']

eval_test_split = eval_test_dataset.train_test_split(test_size=0.5, seed=42)
test_dataset = eval_test_split['train']
eval_dataset = eval_test_split['test']

**Задание 1 (2 балла)**. Реализация LoRA. В этом задании необходимо дообучить LoRA и повысить качество после решения задач. Для выполнения данного задания нужно реализовать следующие пункты: 
1. Посмотрите на данные, которые хранятся в датасете и токенизируйте. Не забудьте, что для обучения модели лучше не использовать токены, соответствующие условию.
1. Реализуйте функцию evaluate_generation, которая замеряет качество модели на тестовом датасете. В качестве метрики качества используйте `accuracy` предсказаний. Используйте few-shot с 3мя примерами решений для того, чтобы модель поняла формат, в котором вы ожидаете ответ.
1. Реализуйте класс `LoRALinear`, который оборачивает модули модели, добавляя обучаемые матрицы для получения предсказаний. Заморозьте модель, оберните модули внимания (`q_proj`, `k_proj`, `v_proj`, `o_proj`)
1. Замерьте качество модели до обучения, обучите модель, замерьте качество после обучения.
1. Сделайте выводы

In [3]:
def tokenize(sample: dict) -> dict:
    """
    Convert a single dataset example into the desired format.

    Args:
        sample (dict): One example containing fields like 'question' and 'answer'.

    Returns:
        dict: A dictionary with the following keys:
            - 'input_ids' (list): Token IDs of the input text.
            - 'attention_mask' (list): Attention mask (1 for tokens to attend to, 0 to ignore).
            - 'labels' (list): Token IDs used for loss computation (can contain -100 for tokens to ignore).
    """
    ### YOUR CODE HERE
    return {'input_ids': [], 'attention_mask': [], 'labels': []}


train_dataset = train_dataset.map(tokenize)
eval_dataset = eval_dataset.map(tokenize)
test_dataset = test_dataset.map(tokenize)

In [6]:
from typing import Union
from transformers import PreTrainedModel, PreTrainedTokenizerBase
from datasets import Dataset

def evaluate_generation(
    model: PreTrainedModel,
    dataset: Dataset,
    tokenizer: PreTrainedTokenizerBase,
    batch_size: int = 4,
    few_shot_size: int = 3
) -> float:
    """
    Evaluate the model's generation accuracy on a dataset.

    Args:
        model (PreTrainedModel): The language model to evaluate.
        dataset (Dataset): The dataset containing examples to generate predictions for.
        tokenizer (PreTrainedTokenizerBase): Tokenizer corresponding to the model.
        batch_size (int, optional): Number of examples per batch. Defaults to 4.
        few_shot_size (int, optional): Number of few-shot examples to prepend. Defaults to 3.

    Returns:
        float: Accuracy of the model's predictions on the dataset.
    """
    ### YOUR CODE HERE
    return 0.0

In [ ]:
import math

class LoRALinear(nn.Module):
    def __init__(self, module, r=8, alpha=16, dropout=0.05):
        super().__init__()
        ### YOUR CODE HERE
        self.module = module
        
    def forward(self, x):
        ### YOUR CODE HERE
        return self.module(x)

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

for param in model.parameters():
    param.requires_grad = False

for name, module in model.named_modules():
    pass
    # Замените все слои, описанные в target_modules на LoRALinear
    ### YOUR CODE HERE


total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

assert total_params == 495114112
assert trainable_params == 1081344

print(f"Percentage of trainable params: {100 * trainable_params / total_params:.2f}%")

In [ ]:
print("Дегенеративная оценка до обучения:")
evaluate_generation(model, test_dataset, tokenizer)

In [ ]:
args = TrainingArguments(
    ### YOUR CODE HERE
)

trainer = Trainer(
    ### YOUR CODE HERE
)

trainer.train()

In [ ]:
print("Дегенеративная оценка после обучения:")
evaluate_generation(model, test_dataset, tokenizer)

**Выводы** (❓):

...

## `Часть 2. Реализация инференса с CPU Offloading (1 балл)`

Вновь начнем с загрузки модели и подготовки референсного предсказания

In [1]:
import re
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch.nn.functional as F

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
input_ids = tokenizer.encode("Hello, world!", return_tensors="pt")
config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)

model = Qwen2ForCausalLM(config)

res_baseline = model(input_ids=input_ids)

Воспользуемся функией проверки того, что все слои на cpu

In [2]:
import torch
from torch import nn

def test_model_on_cpu(model: nn.Module) -> bool:
    """
    Check that all layers in `model.model.layers` are on CPU.

    Args:
        model (nn.Module): The model to check, expected to have `model.model.layers`.

    Returns:
        bool: True if all layers are on CPU, False otherwise.
    """
    for i, layer in enumerate(model.model.layers):
        for name, param in layer.named_parameters():
            if param.device.type != 'cpu':
                return False
    return True

test_model_on_cpu(model)

True

**Задание 2 (1 балл)**. Реализация `CPU Offloading`. Получить инференс модели про помощи простого CPU offloading, в ходе которого необходимый слой грузится с CPU на GPU, получается его предсказание и выгружается обратно. Для выполнения данного задания нужно реализовать следующие пункты: 
1. Реализуйте класс `OffloadBlock`, который оборачивает модули модели и модифицирует их `forward`. Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на GPU.
1. Получите инференс, сравните с референсными значениями


In [ ]:

class OffloadBlock(nn.Module):
    def __init__(self, module: nn.Module, device:str = 'cuda'):
        ### YOUR CODE HERE
        pass

    def forward(self, *args, **kwargs):
        ### YOUR CODE HERE
        # Загружаем на GPU
        # Вызываем forward модуля
        out = ...
        # Выгружаем на CPU
        assert test_model_on_cpu(model)
        return out

In [ ]:
# Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на gpu.
### YOUR CODE HERE

In [ ]:
with torch.no_grad():
    res_offloading = model(input_ids=input_ids.to('cuda'))

assert torch.allclose(res_baseline.logits, res_offloading.logits.cpu(), rtol=0, atol=1e-5), "The outputs of the model before and after offloading do not match."

## `Часть 3. Реализация обучения с CPU Offloading + checkpointing (1 балл)`

Вновь начнем с загрузки модели

In [1]:
import re
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch.nn.functional as F

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
input_ids = tokenizer.encode("Hello, world!", return_tensors="pt")
config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)

model = Qwen2ForCausalLM(config)

Подготовим функцию, которая берет две модели и сравнивает их градиенты

In [5]:
# Если на этом этапе выпадает OOM - можете для дебага уменьшить число слоёв модели.
# config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
# config.num_hidden_layers = 5
# model = Qwen2ForCausalLM(config)

def compare_grads(model1, model2):
    for (n1, p1), (n2, p2) in zip(model1.named_parameters(), model2.named_parameters()):
        assert not (p1.grad is None or p2.grad is None)
        assert torch.allclose(p1.grad.cpu(), p2.grad.cpu(), atol=1e-4)
    print('OK')

**Задание 3 (1 балл)**. Реализация `CPU Offloading` + `Checkpointing`. Обучить модель при помощи CPU offloading и checkpointing, в ходе которого модели модели будут двигаться на gpu и обратно, а промежуточные результаты вычисляться заново при помощи ванильной реализации `checkpointing` из PyTorch. Для выполнения данного задания нужно реализовать следующие пункты: 
1. Реализуйте класс `OffloadBlockWithCheckpointing`, который оборачивает модули модели и модифицирует их `forward`. В ходе `forward` нужно получить предсказания модуля с помощью `checkpoint`. Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на GPU.
1. Возьмите также модель без оффлоадинга.
1. Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.
1. При помощи функции `compare_grads` проверьте, что градиенты после трех шагов совпадают.


In [89]:
from torch.utils.checkpoint import checkpoint

class OffloadBlockWithCheckpointing(nn.Module):
    def __init__(self, module: nn.Module, layer=-1, device='cuda'):
        ### YOUR CODE HERE
        pass
    
    def forward(self, *args, **kwargs):
        ### YOUR CODE HERE
        assert test_model_on_cpu(model)
        return out
	

In [ ]:
# Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на gpu.

In [ ]:
# Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.

In [3]:
compare_grads(model_wo_offloading, model_w_offloading)

## `Часть 4. Реализация обучения при помощи torch.autograd.Function (2 балла)`

Вновь начнем с загрузки модели

In [1]:
import re
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch.nn.functional as F

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
input_ids = tokenizer.encode("Hello, world!", return_tensors="pt")
config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)

model = Qwen2ForCausalLM(config)

Подготовим функцию, которая берет две модели и сравнивает их градиенты

In [5]:
# Если на этом этапе выпадает OOM - можете для дебага уменьшить число слоёв модели.
# config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
# config.num_hidden_layers = 5
# model = Qwen2ForCausalLM(config)

def compare_grads(model1, model2):
    for (n1, p1), (n2, p2) in zip(model1.named_parameters(), model2.named_parameters()):
        assert not (p1.grad is None or p2.grad is None)
        assert torch.allclose(p1.grad.cpu(), p2.grad.cpu(), atol=1e-4)
    print('OK')

**Задание 4 (2 балла)**. Реализация обучения при помощи `torch.autograd.Function`. Обучить модель, реализовав методы `offloading` и `checkpointing` внутри методов функции. Для выполнения данного задания нужно реализовать следующие пункты: 
1. Реализуйте класс `OffloadFunction`, внутри которого содержатся реализации функций `forward` и `backward`. Внутри этих функция должен быть реализован `offloading` и `checkpointing`.
1. Реализуйте класс `OffloadBlockWithFunction`, который оборачивает модули модели и модифицирует их `forward`, применяя функцию `OffloadFunction`. В ходе forward нужно получить предсказания модуля с помощью `checkpoint`. В данном задани пользоваться реализацией `checkpoint` из PyTorch <span style="color:red">запрещено</span>. Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на GPU.
1. Возьмите также модель без оффлоадинга.
1. Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.
1. При помощи функции compare_grads проверьте, что градиенты после трех шагов совпадают.


In [ ]:
class OffloadFunction(torch.autograd.Function):
    @staticmethod
    def forward( ... ):
        ### YOUR CODE HERE
        return outputs if isinstance(outputs, tuple) else (outputs,)

    @staticmethod
    def backward( ... ):
        ### YOUR CODE HERE
        return ...

In [ ]:
class OffloadBlockWithFunction(nn.Module):
    def __init__(self, module: nn.Module, device='cuda'):
        super().__init__()
        ### YOUR CODE HERE

    
    def forward(self, *args, **kwargs):
        ### YOUR CODE HERE
        pass

In [ ]:
# Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на gpu.

In [ ]:
# Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.

In [ ]:
compare_grads(model_wo_offloading, model_w_offloading)

## `Часть 5. Реализация обучения с оффлоадингом и оверлапом (4 балла)`

Ну вы поняли

In [ ]:
import re
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from datasets import load_dataset
import torch.nn.functional as F

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
input_ids = tokenizer.encode("Hello, world!", return_tensors="pt")
config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)

model = Qwen2ForCausalLM(config)

In [ ]:
# Если на этом этапе выпадает OOM - можете для дебага уменьшить число слоёв модели.
# config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
# config.num_hidden_layers = 5
# model = Qwen2ForCausalLM(config)

def compare_grads(model1, model2):
    for (n1, p1), (n2, p2) in zip(model1.named_parameters(), model2.named_parameters()):
        assert not (p1.grad is None or p2.grad is None)
        assert torch.allclose(p1.grad.cpu(), p2.grad.cpu(), atol=1e-4)
    print('OK')

**Задание 5 (3.5 балла)** Реализуйте механизм асинхронного оффлоадинга для модели Qwen2.5. Во время вычислений текущего слоя модель должна заранее подгружать следующий слой и асинхронно выгружать предыдущий, используя несколько CUDA Stream и CUDA Events для синхронизации. Для повышения эффективности необходимо реализовать буфер из трёх слоёв и использовать pinned memory при передаче данных между CPU и GPU.


1. Реализуйте класс `OverlapOffloadFunction`, наследующий `torch.autograd.Function`.
   В нём должны быть реализованы методы `forward` и `backward`, обеспечивающие:
   1. Асинхронную загрузку следующего слоя (prefetch) во время вычислений текущего;
   1. Асинхронную выгрузку предыдущего слоя (offload);
   1. Синхронизацию потоков при помощи `torch.cuda.Event` и `wait_event`;
   1. Использование pinned memory для всех данных, передаваемых между CPU и GPU;
   1. Корректное сохранение необходимых тензоров для `backward`.
<br><br>
1. Класс `OverlapOffloadBlockWithFunction`, оборачивающий блоки трансформера.
   Он должен:
   1. Инициализировать два CUDA Streams — `prefetch_stream` и `offload_stream`;
   1. При каждом шаге:
      * Выполнять вычисления текущего слоя;
      * Загружать следующий слой на GPU в `prefetch_stream`;
      * Выгружать предыдущий слой обратно на CPU в `offload_stream`;
   1. Корректно управлять жизненным циклом потоков и событий (`event.record`, `wait_event`);
   1. Удерживать слои embedding, norm и lm_head постоянно на GPU
<br><br>
1. Возьмите также модель без оффлоадинга.
1. Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.
1. При помощи функции compare_grads проверьте, что градиенты после трех шагов совпадают.


**Практические рекомендации**
1. Можете воспользоваться `torch.profiler` или `nsys` для проверки, что операции `memcpyHtoD` и `kernel` выполняются параллельно.
1. Убедитесь, что pinned memory действительно используется: без неё non_blocking=True не даст эффекта.
1. Не выгружайте слой, пока его тензоры ещё участвуют в вычислениях.
1. Для стабильности можно добавить мягкую синхронизацию: `wait_stream(prefetch_stream)` перед завершением итерации.


In [ ]:
class OffloadFunctionWithOverlap(torch.autograd.Function):
    @staticmethod
    def forward( ... ):
        ### YOUR CODE HERE
        return outputs if isinstance(outputs, tuple) else (outputs,)

    @staticmethod
    def backward( ... ):
        ### YOUR CODE HERE
        return ...

In [ ]:
class OffloadBlockWithOverlapFunction(nn.Module):
    def __init__(self, module: nn.Module, device='cuda'):
        super().__init__()
        ### YOUR CODE HERE

    
    def forward(self, *args, **kwargs):
        ### YOUR CODE HERE
        pass

In [ ]:
# Оберните слои трансформера. Слои с нормализацией, эмбедингами и lm_head всегда держите на gpu.

In [ ]:
# Совершите по три шага обучения для обеих моделей на идентичных входных сэмплах.

In [ ]:
compare_grads(model_wo_offloading, model_w_offloading)

**Задание 6 (0.5 балла)**

Приложите скриншот результатов работы профилировщика, на котором виден overlap загрузки и вычислений

## `Часть 6. Бонусы (3 балла)`

1. (1 балл). Покажите, что написанные вами функции и классы действительно помогают работать с большими моделями. Для этого покажите, что вы умеете работать с `Qwen2.5-7B` и потребляете меньше 10 ГБ видеопамяти, при этом GPU работает.
2. (1 балл). Составьте сводную таблицу, сколько времени занимает forward и backward, а также сколько памяти требует каждый из реализованных вами методов. В качестве бейзлайнов возьмите полностью GPU (если возможно) и полностью CPU реализации.
3. (1 балл). Реализуйте offloading состояния оптимизатора. Проверьте, что использование такой версии оптимизатора приводит к тому, что обучение совпадает с обычной версией.